In [21]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
date = pd.read_pickle('../data/input/date.pkl')

In [3]:
date.head()

,date
0,2011-01-29
1,2011-01-30
2,2011-01-31
3,2011-02-01
4,2011-02-02


In [4]:
train = pd.read_pickle('../data/input/l_train.pkl')

In [5]:
train['DATE'] = date

In [10]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [11]:
train = reduce_mem_usage(train)

Mem. usage decreased to 2614.39 Mb (37.3% reduction)


In [13]:
train['DATE'] = pd.to_datetime(train['DATE'])

In [14]:
train.head()

,target,id,item_id,dept_id,cat_id,store_id,state_id,date,wm_yr_wk,wday,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,DATE
0,0,14370,1437,3,1,0,0,0,11101,1,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,2011-01-29
1,0,14370,1437,3,1,0,0,1,11101,2,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,2011-01-30
2,0,14370,1437,3,1,0,0,2,11101,3,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,2011-01-31
3,0,14370,1437,3,1,0,0,3,11101,4,...,2011,NaN,NaN,NaN,NaN,1,1,0,NaN,2011-02-01
4,0,14370,1437,3,1,0,0,4,11101,5,...,2011,NaN,NaN,NaN,NaN,1,0,1,NaN,2011-02-02


In [17]:
date_all = train['DATE'].unique()

In [32]:
date_all.max() - np.timedelta64(28,'D')

numpy.datetime64('2016-03-27T00:00:00.000000000')

In [37]:
df_target = train['target']

In [38]:
df_target.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int16

In [40]:
for i in range(5):
    val_date = date_all.max() - np.timedelta64(28,'D')
    X_val = train[(train['DATE']>val_date)].drop('DATE',axis=1)
    X_train = train[(train['DATE']<=val_date)].drop('DATE',axis=1)

    y_val = df_target.iloc[X_val.index]
    y_train = df_target.iloc[X_train.index]

In [69]:
val_date_list = []
for i in range(2):
    if i == 0:
        val_date = date_all.max() - np.timedelta64(28,'D')
        val_date_list.append(val_date)
        X_val = train[(train['DATE']>val_date)].drop('DATE',axis=1)
        X_train = train[(train['DATE']<=val_date)].drop('DATE',axis=1)
    else:
        val_date = val_date - np.timedelta64(28,'D')
        val_date_list.append(val_date)
        X_val = train[(train['DATE']>val_date_list[i]) & (train['DATE']<=val_date_list[i-1])].drop('DATE',axis=1)
        X_train = train[(train['DATE']<=val_date)].drop('DATE',axis=1)

    y_val = df_target.iloc[X_val.index]
    y_train = df_target.iloc[X_train.index]

In [70]:
len(X_val['id'])/X_val['id'].nunique()

28.0

In [60]:
val_date_list = []
for i in range(2):
    if i == 0:
        val_date = date_all.max() - np.timedelta64(28,'D')
        val_date_list.append(val_date)
    else:
        val_date = val_date - np.timedelta64(28,'D')
        val_date_list.append(val_date)

In [71]:
train = reduce_mem_usage(train)

Mem. usage decreased to 5174.39 Mb (0.0% reduction)


In [72]:
train.to_pickle('../data/input/dl_train.pkl')

In [55]:
date_all.max()

numpy.datetime64('2016-04-24T00:00:00.000000000')